In [19]:
import torch
from torch import nn
from einops import rearrange

In [5]:
# weight = torch.randn([2304, 768])

In [6]:
linear = nn.Linear(768, 768*3)

In [7]:
linear.weight.shape

torch.Size([2304, 768])

In [8]:
linear.bias.shape

torch.Size([2304])

In [9]:
input = torch.randn([16, 128, 768])

In [12]:
(input @ linear.weight.T).shape

torch.Size([16, 128, 2304])

In [16]:
ref_output = input @ linear.weight.T + linear.bias

In [20]:
batch_size = input.shape[0]
seq_len = input.shape[1]
input = rearrange(input, "b n h -> (b n) h")

In [21]:
input.shape

torch.Size([2048, 768])

In [24]:
(input @ linear.weight.T).shape

torch.Size([2048, 2304])

In [26]:
input.shape

torch.Size([2048, 768])

In [27]:
input.shape

torch.Size([2048, 768])

In [2]:
import torch

In [24]:
args = [
(
    torch.randn(16, 32, dtype=torch.float32),
    torch.randn(16, 32, dtype=torch.float32),
    False,
    True,
),
(
    torch.randn(16, 32, dtype=torch.float32),
    torch.randn(16, 32, dtype=torch.float32),
    True,
    False,
),
(
    torch.randn(32, 16, dtype=torch.float32),
    torch.randn(64, 32, dtype=torch.float32),
    True,
    True,
),
(
    torch.randn(32, 16, dtype=torch.float32),
    torch.randn(16, 64, dtype=torch.float32),
    False,
    False,
)
]

In [25]:
def normal_matmul(a, transpose_a, b, transpose_b):
    a = a.T if transpose_a is True else a
    b = b.T if transpose_b is True else b
    return torch.matmul(a, b)

In [26]:
# # orig
# def fp8_matmul(a, transpose_a, b, transpose_b):
#     a = a.T if transpose_a is False else a
#     b = b.T if transpose_b is True else b
#     return torch.matmul(a.T, b)

In [33]:
# orig
def fp8_matmul_2(a, transpose_a, b, transpose_b):

    if transpose_a is True:
        a = a
    elif transpose_a is False:
        a = a.T

    if transpose_b is False:
        b = b
    elif transpose_b is True:
        b = b.T

    predicted_shape = torch.zeros(a.T.shape[0], b.shape[-1]).shape
    
    # a = a if transpose_a is True else a
    # b = b.T if transpose_b is True else b
    output = torch.matmul(a.T, b)
    assert predicted_shape == output.shape
    return output

In [34]:
for (a, b, transpose_a, transpose_b) in args:
    ref_output = normal_matmul(a, transpose_a, b, transpose_b)
    output = fp8_matmul_2(a, transpose_a, b, transpose_b)
    
    print(torch.equal(output, ref_output))

True
True
True
True


In [35]:
x, y, _, _ = args[0]

In [41]:
import torch.nn.functional as F

In [43]:
x @ y.T == F.linear(x, y)

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,